In [2]:
import sys
sys.path.append('./lagrangian-dual-deep-learning/transprecision_computing/')
from dataset import *
from Regressor import   *
from SBRregressor import  *
from SBRregressor2 import  *
from settings import *
from util import *
import argparse
import torch
Ten = torch.FloatTensor
iTen = torch.LongTensor

In [ ]:
%cd /lagrangian-dual-deep-learning/transprecision_computing/


### 1.  Specify main parameters for running experiments


In [15]:
test_seed = 0
violated_const_ratio = 0.5 # you can adjust this number to play around 
start_point_seed = 2

benchmark = 'convolution'

params = {'epochs': 150,
               'n_data': 4000,
               'batch_size': 256,
               'violated_const_ratio': violated_const_ratio,  # this is used to create a trainig set with a specific
               'benchmark': benchmark,
               'split': [0.5, 0.25, 0.25],
               'seed': test_seed}

d_trainall = Dataset(params, 'train', 'cpu')
d_test = Dataset(params, 'test', 'cpu')
d_valall = Dataset(params, 'valid', 'cpu')
res['d_test'] = d_test
X_test, y_test = d_test._dataset
train_size = 2000 # more training size models will be more accurate
val_size = 600 
split_seed = 0 

### 2. Get train/val dataset 

In [ ]:
# Based on the above defined parameters, we then  can have train/ validation dataset
# In our experiments, we fixed the test set
# We increase the training size gradually to see its impact 

X_train,y_train  = copy.deepcopy( d_trainall._dataset)
idx_train = np.random.choice( list(range(len(X_train))), train_size)
X_train, y_train = X_train[idx_train,:], y_train[idx_train]

d_train = copy.deepcopy(d_trainall)
d_train._dataset = (X_train, y_train)
y_med_pred = np.median(y_train)* np.ones(len(y_test))

X_val, y_val = copy.deepcopy(d_valall._dataset)
idx_val = np.random.choice(list(range(len(X_val))), val_size)
X_val, y_val = X_val[idx_val,:], y_val[idx_val]
d_val = copy.deepcopy(d_valall)
d_val._dataset = (X_val, y_val)

### 3. Run all models

In [16]:
# A simple regressor without constraints
model_1 = Regressor(params, d_train, d_test, d_val,  start_point_seed) # start_point_seed is random seed of pytorch
# make sure all models starting from the same initial points
model_1.train()
tmp = model_1.test()
print('Performance of a simple regressor ')
print ('MAE = {}, Constraint Violation = {}'.format(tmp[0], tmp[1]))
print('-------------------------')



# print('Performance of a simple regressor ')

model_2_1 = SBRregressor(params, d_train, d_test, d_val,  start_point_seed)
model_2_1.train(options = {'mult_fixed':True})

tmp = model_2_1.test()
print('Performance of a constrained regressor, but we fix the multiplier during the course of training ')
print ('MAE = {}, Constraint Violation = {}'.format(tmp[0], tmp[1]))

print('-------------------------')

###################regularization with single multiplier updated gradually, starts with 0

model_2 = SBRregressor(params, d_train, d_test, d_val,  start_point_seed)
if split_seed == 0:
    model_2.opt_lr_rate()
    best_lr_model_2 = copy.deepcopy( model_2._LR_rate)
else:
    model_2._LR_rate = copy.deepcopy( best_lr_model_2)
    model_2.train(options={'mult_fixed': False})
tmp = model_2.test()


print('Performance of a constrained regressor, using single/common multiplier for all constraints, updating the multiplier by epochs')
print ('MAE = {}, Constraint Violation = {}'.format(tmp[0], tmp[1]))

print('-------------------------')



########## regularization with a multiplier for each constraint, each multiplier has updated gradually, starts with 0

# Note the difference between model_3 vs model_2 is in model_3 we have different multpliers instead of using shared multiplier
# as in model_2
model_3 = SBRregressor2(params, d_train, d_test, d_val,  start_point_seed)
if split_seed == 0:
    model_3.opt_lr_rate()
    best_lr_model_3 = copy.deepcopy(model_3._LR_rate)
else:
    model_3._LR_rate = copy.deepcopy( best_lr_model_3)
    model_3.train(options={'mult_fixed': False})

tmp = model_3.test()


print('Constrained Regressor, using different multipliers for all constraints, updating all multipliers by epochs')
print ('MAE = {}, Constraint Violation = {}'.format(tmp[0], tmp[1]))

print('-------------------------')


Performance of a simple regressor 
MAE = 0.0747935026884079, Constraint Violation = 198
-------------------------
Performance of a constrained regressor, but we fix the multiplier during the course of training 
MAE = 0.13169240951538086, Constraint Violation = 0
-------------------------
Performance of a constrained regressor, using single/common multiplier for all constraints, updating the multiplier by epochs
MAE = 0.09749921411275864, Constraint Violation = 4
-------------------------
Constrained Regressor, using different multipliers for all constraints, updating all multipliers by epochs
MAE = 0.07603932917118073, Constraint Violation = 17
-------------------------
